In [9]:
import glob
from PIL import Image
#import PIL.Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from tqdm import tqdm
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,  Activation, Flatten
from keras.layers import Merge
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.applications.inception_v3 import InceptionV3# .    for pretrained CNN
from keras.preprocessing import image
import nltk

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ImportError: cannot import name 'Merge'

In [ ]:
token = 'Flickr8k_text/Flickr8k.token.txt'

In [ ]:
captions = open(token, 'r').read().strip().split('\n')

## Creating a dictionary containing all the captions of the images

In [10]:
d = {}#dictionary containing all the captions of the images

for i, row in enumerate(captions):
    
    row = row.split('\t')
    
    row[0] = row[0][:len(row[0])-2]
    if row[0] in d:
        d[row[0]].append(row[1])
    else:
        d[row[0]] = [row[1]]


NameError: name 'captions' is not defined

In [11]:
d['1000268201_693b08cb0e.jpg']

KeyError: '1000268201_693b08cb0e.jpg'

In [12]:
images = 'Flicker8k_Dataset/'

In [13]:
# Contains all the images
img = glob.glob(images+'*.jpg')
# return a list or an iterator of files in a directory using shell pattern matching.

In [14]:
img[:5]

['Flicker8k_Dataset/2387197355_237f6f41ee.jpg',
 'Flicker8k_Dataset/2609847254_0ec40c1cce.jpg',
 'Flicker8k_Dataset/2046222127_a6f300e202.jpg',
 'Flicker8k_Dataset/2853743795_e90ebc669d.jpg',
 'Flicker8k_Dataset/2696951725_e0ae54f6da.jpg']

In [15]:
train_images_file = 'Flickr8k_text/Flickr_8k.trainImages.txt'

In [16]:
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

In [17]:
def split_data(l):
    temp = []
    for i in img:
        if i[len(images):] in l:# if actal images actally have themseleves registered in test image entries
            temp.append(i)
            #print(i)
    return temp

In [18]:
# Getting the training images from all the images
train_img = split_data(train_images)
len(train_img)

6000

In [19]:
val_images_file = 'Flickr8k_text/Flickr_8k.devImages.txt'
val_images = set(open(val_images_file, 'r').read().strip().split('\n'))# set form text files

In [20]:
# Getting the validation images from all the images
val_img = split_data(val_images)
len(val_img)
#print(val_img)

1000

In [21]:
test_images_file = 'Flickr8k_text/Flickr_8k.testImages.txt'
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))

In [22]:
# Getting the testing images from all the images
test_img = split_data(test_images)
len(test_img)

1000

In [23]:
#Image.open(train_img[0])

In [24]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [25]:
def preprocess(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    x = preprocess_input(x)
    return x

In [26]:
#plt.imshow(np.squeeze(preprocess(train_img[0])))

In [27]:
model = InceptionV3(weights='imagenet')

NameError: name 'InceptionV3' is not defined

In [28]:
from keras.models import Model

new_input = model.input
hidden_layer = model.layers[-2].output

model_new = Model(new_input, hidden_layer)# the first and last layer

NameError: name 'model' is not defined

In [29]:
tryi = model_new.predict(preprocess(train_img[0]))

NameError: name 'model_new' is not defined

In [30]:
tryi.shape

NameError: name 'tryi' is not defined

In [31]:
def encode(image):
    image = preprocess(image)
    temp_enc = model_new.predict(image)
    temp_enc = np.reshape(temp_enc, temp_enc.shape[1])
    return temp_enc

In [32]:
encoding_train = {}
for img in tqdm(train_img):
    encoding_train[img[len(images):]] = encode(img)

  0%|          | 0/6000 [00:00<?, ?it/s]


NameError: name 'image' is not defined

In [33]:
print(encoding_train['2387197355_237f6f41ee.jpg'])
encoding_train['2387197355_237f6f41ee.jpg'].shape

KeyError: '2387197355_237f6f41ee.jpg'

In [34]:
with open("encoded_images_inceptionV3.p", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle) 

In [35]:
encoding_train = pickle.load(open('encoded_images_inceptionV3.p', 'rb'))

In [36]:
encoding_train['3556792157_d09d42bef7.jpg'].shape

KeyError: '3556792157_d09d42bef7.jpg'

In [37]:
for img in tqdm(test_img):
    print(img[len(images):] )
    #print(img)

100%|██████████| 1000/1000 [00:00<00:00, 13649.25it/s]

2869491449_1041485a6b.jpg
2924259848_effb4dcb82.jpg
2300168895_a9b83e16fc.jpg
3530843182_35af2c821c.jpg
3439382048_d2e23b2b4c.jpg
3592992234_6d3fe58a70.jpg
3568197730_a071d7595b.jpg
2843695880_eeea6c67db.jpg
3030294889_78b2ccbe51.jpg
2901074943_041aba4607.jpg
3354883962_170d19bfe4.jpg
3062173277_bfb5ef4c45.jpg
1317292658_ba29330a0b.jpg
498444334_a680d318a1.jpg
3393926562_66cc01b001.jpg
3480051754_18e5802558.jpg
3542484764_77d8920ec9.jpg
160792599_6a7ec52516.jpg
2594042571_2e4666507e.jpg
2689001252_e0016c89f0.jpg
3200120942_59cfbb3437.jpg
3047264346_e24601bfbf.jpg
2196107384_361d73a170.jpg
2698666984_13e17236ae.jpg
2883099128_0b056eed9e.jpg
2599444370_9e40103027.jpg
540721368_12ac732c6c.jpg
2208067635_39a03834ca.jpg
3477715432_79d82487bb.jpg
3139876823_859c7d7c23.jpg
1836335410_de8313a64e.jpg
2078311270_f01c9eaf4c.jpg
544576742_283b65fa0d.jpg
3334537556_a2cf4e9b9a.jpg
2077079696_03380d218b.jpg
138718600_f430ebca17.jpg
452419961_6d42ab7000.jpg
2140182410_8e2a06fbda.jpg
3245912109_fdeef6b

In [38]:
encoding_test = {}
for img in tqdm(test_img):
    encoding_test[img[len(images):]] = encode(img)

  0%|          | 0/1000 [00:00<?, ?it/s]


NameError: name 'image' is not defined

In [39]:
with open("encoded_images_test_inceptionV3.p", "wb") as encoded_pickle:
    pickle.dump(encoding_test, encoded_pickle) 

In [40]:
encoding_test = pickle.load(open('encoded_images_test_inceptionV3.p', 'rb'))

In [41]:
encoding_test[test_img[0][len(images):]].shape

KeyError: '2869491449_1041485a6b.jpg'

In [42]:
train_d = {}
for i in train_img:
    if i[len(images):] in d:# d = {}#dictionary containing all the captions of the images
        train_d[i] = d[i[len(images):]]

In [43]:
len(train_d)

0

In [44]:
train_d[images+'3556792157_d09d42bef7.jpg']# images = 'Flicker8k_Dataset/'

KeyError: 'Flicker8k_Dataset/3556792157_d09d42bef7.jpg'

In [45]:
val_d = {}
for i in val_img:
    if i[len(images):] in d:
        val_d[i] = d[i[len(images):]]

In [46]:
len(val_d)

0

In [47]:
test_d = {}
for i in test_img:
    if i[len(images):] in d:
        test_d[i] = d[i[len(images):]]

In [48]:
len(test_d)

0

Calculating the unique words in the vocabulary.

In [49]:
caps = []
for key, val in train_d.items():
    for i in val:
        caps.append('<start> ' + i + ' <end>')
print(caps)       

[]


In [50]:
words = [i.split() for i in caps]


In [51]:
unique = []
for i in words:
    unique.extend(i)

In [52]:
unique = list(set(unique))
len(unique)

0

In [53]:
 with open("unique.p", "wb") as pickle_d:
    pickle.dump(unique, pickle_d) 

In [54]:
unique = pickle.load(open('unique.p', 'rb'))

In [55]:
#len(unique)
print(unique)

[]


Mapping the unique words to indices and vice-versa

In [56]:
word2idx = {val:index for index, val in enumerate(unique)}

In [57]:
print(word2idx['<start>'])
word2idx['subway']

KeyError: '<start>'

In [58]:
idx2word = {index:val for index, val in enumerate(unique)}

In [59]:
idx2word[5553]

KeyError: 5553

Calculating the maximum length among all the captions

In [60]:
max_len = 0
for c in caps:
    c = c.split()
    if len(c) > max_len:
        max_len = len(c)
max_len

0

In [61]:
len(unique), max_len

(0, 0)

In [62]:
vocab_size = len(unique)

In [63]:
vocab_size

0

Adding <start> and <end> to all the captions to indicate the starting and ending of a sentence. This will be used while we predict the caption of an image

In [64]:
f = open('flickr8k_training_dataset.txt', 'w')
f.write("image_id\tcaptions\n")

18

In [65]:
for key, val in train_d.items():
    for i in val:
        f.write(key[len(images):] + "\t" + "<start> " + i +" <end>" + "\n")

f.close()

In [66]:
df = pd.read_csv('flickr8k_training_dataset.txt', delimiter='\t')

In [67]:
df[:5]

,image_id,captions


In [68]:
len(df)

0

In [69]:
df.describe()

,image_id,captions
count,0,0
unique,0,0


In [70]:
df['captions'][0]

IndexError: index out of bounds

In [71]:
c = [i for i in df['captions']]
len(c)
c[0]

IndexError: list index out of range

In [72]:
imgs = [i for i in df['image_id']]
imgs[0]

IndexError: list index out of range

In [73]:
a = c[-1]
a, imgs[-1]

IndexError: list index out of range

In [74]:
for i in a.split():
    print (i, "=>", word2idx[i])

NameError: name 'a' is not defined

In [75]:
samples_per_epoch = 0
for ca in caps:# caps is number of training captions
    #print(ca)
    samples_per_epoch += len(ca.split())-1

In [76]:
trya='<start> A group of people are partying at a masquerade party . <end>'
print(trya.split())

['<start>', 'A', 'group', 'of', 'people', 'are', 'partying', 'at', 'a', 'masquerade', 'party', '.', '<end>']


In [77]:
samples_per_epoch

0

In [78]:
lf = df.sample(frac=1)

ValueError: a must be greater than 0

In [79]:
print(lf)

NameError: name 'lf' is not defined

## Generator 

We will use the encoding of an image and use a start word to predict the next word.
After that, we will again use the same image and use the predicted word 
to predict the next word.
So, the image will be used at every iteration for the entire caption. 
This is how we will generate the caption for an image. Hence, we need to create 
a custom generator for that.

The CS231n lecture by Andrej Karpathy explains this concept very clearly and beautifully.
Link for the lecture:- https://youtu.be/cO0a0QYmFm8?t=32m25s

In [80]:
iter = df.iterrows()

In [81]:
for i in range(df.shape[0]):
            x = next(iter)
            
            sampsent=(x[1][1].split())
            sampname=(x[1][0])


In [82]:
s=str(sampsent).split()
s

NameError: name 'sampsent' is not defined

In [83]:
for i in range(len(s)-1):
    print(s[:i+1])

NameError: name 's' is not defined

In [84]:
for i in range(len(s)-1):
    print(word2idx[j] for j in s[:i+1])


NameError: name 's' is not defined

In [85]:
def data_generator(batch_size = 32):
        
        next_words = []
        images = []
        partial_caps = []
        df = pd.read_csv('flickr8k_training_dataset.txt', delimiter='\t')
        df = df.sample(frac=1)
        iter = df.iterrows()
        c = []
        imgs = []
        for i in range(df.shape[0]):
            x = next(iter)
            
            c.append(x[1][1])# captions
            imgs.append(x[1][0])# image name


        count = 0
        while True:
            for j, text in enumerate(c):
                current_image = encoding_train[imgs[j]]# the extracted features
                for i in range(len(text.split())-1):
                    count+=1
                    # for each word in each caption
                    partial = [word2idx[txt] for txt in text.split()[:i+1]]
                    partial_caps.append(partial)
                    
                    # Initializing with zeros to create a one-hot encoding matrix
                    # This is what we have to predict
                    # Hence initializing it with vocab_size length
                    n = np.zeros(vocab_size)# 8256
                    # Setting the next word to 1 in the one-hot encoded matrix
                    n[word2idx[text.split()[i+1]]] = 1
                    next_words.append(n)
                    
                    images.append(current_image)# encoding of currnet image

                    if count>=batch_size:
                        next_words = np.asarray(next_words)
                        images = np.asarray(images)
                        partial_caps = sequence.pad_sequences(partial_caps, maxlen=max_len, padding='post')
                        #This class takes in a sequence of data-points gathered at equal intervals,
                        #along with time series parameters such as stride, 
                        #length of history, etc., to produce batches for training/validation.


                        yield [[images, partial_caps], next_words] # a replacement for return
                        partial_caps = []
                        next_words = []
                        images = []
                        count = 0

In [86]:
#The yield statement suspends function’s execution and sends a value back to caller, 
#but retains enough state to enable function to resume where it is left off. 
#When resumed, the function continues execution immediately after the last yield run. 
#This allows its code to produce a series of values over time, rather them computing them at once
#and sending them back like a list.

## Let's create the model

In [87]:
embedding_size = 300

In [88]:
Sequential?

In [89]:
#sequntial means single input and single output 
# the first argument is what dimensional vector its gonna spit out

Input dimension is 4096 since we will feed it the encoded version of the image.

In [90]:
image_model = Sequential([
        Dense(embedding_size, input_shape=(2048,), activation='relu'),# each word in represented by 300 vectors
        RepeatVector(max_len)# Repeats the input n times. max len any caption can have ONLY THE LAST LAYER
    ])
image_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               614700    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 0, 300)            0         
Total params: 614,700
Trainable params: 614,700
Non-trainable params: 0
_________________________________________________________________


Since we are going to predict the next word using the previous words(length of previous words changes with every iteration over the caption), we have to set return_sequences = True.

In [91]:
caption_model = Sequential([
    # outputs a voab_size because this embedding has a relation to every other word in the vocab
        Embedding(vocab_size, embedding_size, input_length=max_len),
        LSTM(256, return_sequences=True),
        TimeDistributed(Dense(300))
    ])
caption_model.summary()
# EMBEDDING
#Turns positive integers (indexes) into dense vectors of fixed size. eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]
#This layer can only be used as the first layer in a model.
#Sparse vectors are when you have a lot of values in the vector as zero.
#While a dense vector is when most of the values in the vector are non zero.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 0, 300)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 0, 256)            570368    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 0, 300)            77100     
Total params: 647,468
Trainable params: 647,468
Non-trainable params: 0
_________________________________________________________________


In [92]:
#Consider a batch of 32 samples, where each sample is a sequence of 10 vectors of 16 dimensions. 
#The batch input shape of the layer is then (32, 10, 16), and the input_shape, not including the samples dimension,
# is  (10, 16).
#You can then use TimeDistributed to apply a Dense layer to each of the 10 timesteps, independently:

## as the first layer in a model
#model = Sequential()
#model.add(TimeDistributed(Dense(8), input_shape=(10, 16)))
# now model.output_shape == (None, 10, 8)
#The output will then have shape (32, 10, 8).


Merging the models and creating a softmax classifier

In [93]:
final_model = Sequential([
        Merge([image_model, caption_model], mode='concat', concat_axis=1),
        Bidirectional(LSTM(256, return_sequences=False)),
        Dense(vocab_size),
        Activation('softmax')
    ])
#In problems where all timesteps of the input sequence are available, 
#Bidirectional LSTMs train two instead of one LSTMs on the input sequence.
#The first on the input sequence as-is and the second on a reversed copy of the input sequence. 
#This can provide additional context to the network and result in faster and even fuller learning on the problem.


NameError: name 'Merge' is not defined

In [94]:
final_model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

NameError: name 'final_model' is not defined

In [95]:
final_model.summary()

NameError: name 'final_model' is not defined

In [96]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [97]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [98]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [99]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [100]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [101]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [102]:
final_model.optimizer.lr = 1e-4
#final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
 #                         verbose=2)

NameError: name 'final_model' is not defined

In [103]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [104]:
final_model.save_weights('time_inceptionV3_7_loss_3.2604.h5')

NameError: name 'final_model' is not defined

In [105]:
final_model.load_weights('time_inceptionV3_7_loss_3.2604.h5')

NameError: name 'final_model' is not defined

In [106]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [107]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [108]:
final_model.save_weights('time_inceptionV3_3.21_loss.h5')

NameError: name 'final_model' is not defined

In [109]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [110]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [111]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [112]:
final_model.save_weights('time_inceptionV3_3.15_loss.h5')

NameError: name 'final_model' is not defined

In [113]:
final_model.fit_generator(data_generator(batch_size=128), samples_per_epoch=samples_per_epoch, nb_epoch=1, 
                          verbose=2)

NameError: name 'final_model' is not defined

In [114]:
#final_model.load_weights('time_inceptionV3_1.5987_loss.h5')   
final_model.load_weights('time_inceptionV3_3.15_loss.h5')

NameError: name 'final_model' is not defined

## Predict funtion

In [115]:
def predict_captions(image):
    print(image)
    start_word = ["<start>"]
    while True:
        par_caps = [word2idx[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=max_len, padding='post')
       # print(par_caps)
        e = encoding_test[image[len(images):]]
        
        
        preds = final_model.predict([np.array([e]), np.array(par_caps)])
       # PREDICTING USING BOTH THE IMAGE ENCODING AND THE SENTECE EMBEDDING
        
        word_pred = idx2word[np.argmax(preds[0])]# (1,8256)  Returns the indices of the maximum values along an axis.
        print(word_pred)
        start_word.append(word_pred)
        # Returns the indices of the maximum values along an axis.

        
        if word_pred == "<end>" or len(start_word) > max_len:
            break
            
    return ' '.join(start_word[1:-1])

In [116]:
random_images_file = 'Flickr8k_text/Flickr_8k.randomImages.txt'
random_images = set(open(random_images_file, 'r').read().strip().split('\n'))
random_images

{'oms.jpg'}

In [117]:
def split_data(l):
    temp = []
    for i in img:
        if i[len(images):] in l:
            print(i[len(images):])
            temp.append(i)
    return temp

In [118]:
# Getting the testing images from all the images
random_img = split_data(random_images)
len(random_img)


0

In [119]:
def encode(image):
    image = preprocess(image)
    temp_enc = model_new.predict(image)
    temp_enc = np.reshape(temp_enc, temp_enc.shape[1])
    return temp_enc

In [120]:
#####################################
encoding_random_image = {}
for img in tqdm(random_img):
    encoding_random_image[img[len(images):]] = encode(img)

0it [00:00, ?it/s]


In [121]:
#####################################


In [122]:
################################
def predict_captions2(image):
    
    start_word = ["<start>"]
    while True:
        par_caps = [word2idx[i] for i in start_word]
        par_caps = sequence.pad_sequences([par_caps], maxlen=max_len, padding='post')
        e = encoding_random_image[image[len(images):]]
        #This function transforms a list of num_samples sequences (lists of integers) into a 2D Numpy array of shape 
        #(num_samples, num_timesteps). 
        #num_timesteps is either the maxlen argument if provided, or the length of the longest sequence otherwise.

        preds = final_model.predict([np.array([e]), np.array(par_caps)])
        
        word_pred = idx2word[np.argmax(preds[0])]
        start_word.append(word_pred)
        
        if word_pred == "<end>" or len(start_word) > max_len:
            break
            
    return ' '.join(start_word[1:-1])

In [123]:
def beam_search_predictions(image, beam_index = 3):
    start = [word2idx["<start>"]]
    
    start_word = [[start, 0.0]]
    
    while len(start_word[0][0]) < max_len:
        temp = []
        for s in start_word:
            par_caps = sequence.pad_sequences([s[0]], maxlen=max_len, padding='post')
            e = encoding_test[image[len(images):]]
            preds = final_model.predict([np.array([e]), np.array(par_caps)])
            
            word_preds = np.argsort(preds[0])[-beam_index:]
            
            # Getting the top <beam_index>(n) predictions and creating a 
            # new list so as to put them via the model again
            for w in word_preds:
                next_cap, prob = s[0][:], s[1]
                next_cap.append(w)
                prob += preds[0][w]
                temp.append([next_cap, prob])
                    
        start_word = temp
        # Sorting according to the probabilities
        start_word = sorted(start_word, reverse=False, key=lambda l: l[1])
        # Getting the top words
        start_word = start_word[-beam_index:]
    
    start_word = start_word[-1][0]
    intermediate_caption = [idx2word[i] for i in start_word]

    final_caption = []
    
    for i in intermediate_caption:
        if i != '<end>':
            final_caption.append(i)
        else:
            break
    
    final_caption = ' '.join(final_caption[1:])
    return final_caption

In [124]:
print(random_img[0])

IndexError: list index out of range

In [125]:
try_image = random_img[0]
print(try_image)
Image.open(try_image)

IndexError: list index out of range

In [126]:
a=predict_captions2(try_image) 
print(a)


#print ('Beam Search, k=3:', beam_search_predictions(try_image, beam_index=3))
#print ('Beam Search, k=5:', beam_search_predictions(try_image, beam_index=5))
#print ('Beam Search, k=7:', beam_search_predictions(try_image, beam_index=7))
a="I think the thing you are looking at is  "+a
tts=gTTS(text=a,lang='en')
tts.save("generated.mp3")
Audio("generated.mp3", autoplay=True)

NameError: name 'try_image' is not defined

In [127]:
from gtts import gTTS
import os

ModuleNotFoundError: No module named 'gtts'

In [128]:
a="solved !"

In [129]:
tts=gTTS(text=a,lang='en')
tts.save("generated.mp3")

NameError: name 'gTTS' is not defined

In [130]:
from IPython.display import *
Audio("generated.mp3", autoplay=True)

ValueError: could not convert string to float: 'generated.mp3'

In [131]:
try_image2 = test_img[7]
Image.open(try_image2)

AttributeError: type object 'Image' has no attribute 'open'

In [132]:
a= predict_captions(try_image2)
#print ('Beam Search, k=3:', beam_search_predictions(try_image2, beam_index=3))
#print ('Beam Search, k=5:', beam_search_predictions(try_image2, beam_index=5))
#print ('Beam Search, k=7:', beam_search_predictions(try_image2, beam_index=7))
a="I think the thing you are looking at is  "+a
tts=gTTS(text=a,lang='en')
tts.save("generated.mp3")
Audio("generated.mp3", autoplay=True)

Flicker8k_Dataset/2843695880_eeea6c67db.jpg


KeyError: '<start>'

In [133]:
try_image3 = test_img[851]
print(test_img[851])
#Image.open(try_image3)

Flicker8k_Dataset/3053743109_a2d780c0d2.jpg


In [134]:
print ('Normal Max search:', predict_captions(try_image3)) 
print ('Beam Search, k=3:', beam_search_predictions(try_image3, beam_index=3))
print ('Beam Search, k=5:', beam_search_predictions(try_image3, beam_index=5))
print ('Beam Search, k=7:', beam_search_predictions(try_image3, beam_index=7))

Flicker8k_Dataset/3053743109_a2d780c0d2.jpg


KeyError: '<start>'

In [135]:
try_image4 = 'Flicker8k_Dataset/136552115_6dc3e7231c.jpg'
print ('Normal Max search:', predict_captions(try_image4))
print ('Beam Search, k=3:', beam_search_predictions(try_image4, beam_index=3))
print ('Beam Search, k=5:', beam_search_predictions(try_image4, beam_index=5))
print ('Beam Search, k=7:', beam_search_predictions(try_image4, beam_index=7))
Image.open(try_image4)

Flicker8k_Dataset/136552115_6dc3e7231c.jpg


KeyError: '<start>'

In [136]:
im = 'Flicker8k_Dataset/1674612291_7154c5ab61.jpg'
print ('Normal Max search:', predict_captions(im))
print ('Beam Search, k=3:', beam_search_predictions(im, beam_index=3))
print ('Beam Search, k=5:', beam_search_predictions(im, beam_index=5))
print ('Beam Search, k=7:', beam_search_predictions(im, beam_index=7))
Image.open(im)

Flicker8k_Dataset/1674612291_7154c5ab61.jpg


KeyError: '<start>'

In [137]:
im = 'Flicker8k_Dataset/384577800_fc325af410.jpg'
print ('Normal Max search:', predict_captions(im))
#print ('Beam Search, k=3:', beam_search_predictions(im, beam_index=3))
#print ('Beam Search, k=5:', beam_search_predictions(im, beam_index=5))
#print ('Beam Search, k=7:', beam_search_predictions(im, beam_index=7))
Image.open(im)

Flicker8k_Dataset/384577800_fc325af410.jpg


KeyError: '<start>'

In [138]:
im = 'newPhotos/a.jpeg'
#print ('Normal Max search:', predict_captions(im))
#print ('Beam Search, k=3:', beam_search_predictions(im, beam_index=3))
#print ('Beam Search, k=5:', beam_search_predictions(im, beam_index=5))
#print ('Beam Search, k=7:', beam_search_predictions(im, beam_index=7))
Image.open(im)

AttributeError: type object 'Image' has no attribute 'open'

In [139]:
im = 'Flicker8k_Dataset/3631986552_944ea208fc.jpg'
print ('Normal Max search:', predict_captions(im))
#print ('Beam Search, k=3:', beam_search_predictions(im, beam_index=3))
#print ('Beam Search, k=5:', beam_search_predictions(im, beam_index=5))
#print ('Beam Search, k=7:', beam_search_predictions(im, beam_index=7))
Image.open(im)

Flicker8k_Dataset/3631986552_944ea208fc.jpg


KeyError: '<start>'

In [140]:
im = 'Flicker8k_Dataset/3320032226_63390d74a6.jpg'
print ('Normal Max search:', predict_captions(im))
print ('Beam Search, k=3:', beam_search_predictions(im, beam_index=3))
print ('Beam Search, k=5:', beam_search_predictions(im, beam_index=5))
print ('Beam Search, k=7:', beam_search_predictions(im, beam_index=7))
Image.open(im)

Flicker8k_Dataset/3320032226_63390d74a6.jpg


KeyError: '<start>'

In [141]:
im = 'Flicker8k_Dataset/3316725440_9ccd9b5417.jpg'
print ('Normal Max search:', predict_captions(im))
print ('Beam Search, k=3:', beam_search_predictions(im, beam_index=3))
print ('Beam Search, k=5:', beam_search_predictions(im, beam_index=5))
print ('Beam Search, k=7:', beam_search_predictions(im, beam_index=7))
Image.open(im)

Flicker8k_Dataset/3316725440_9ccd9b5417.jpg


KeyError: '<start>'

In [142]:
im = 'Flicker8k_Dataset/2306674172_dc07c7f847.jpg'
print ('Normal Max search:', predict_captions(im))
print ('Beam Search, k=3:', beam_search_predictions(im, beam_index=3))
print ('Beam Search, k=5:', beam_search_predictions(im, beam_index=5))
print ('Beam Search, k=7:', beam_search_predictions(im, beam_index=7))
Image.open(im)

Flicker8k_Dataset/2306674172_dc07c7f847.jpg


KeyError: '<start>'

In [143]:
im = 'Flicker8k_Dataset/2542662402_d781dd7f7c.jpg'
print ('Normal Max search:', predict_captions(im))
print ('Beam Search, k=3:', beam_search_predictions(im, beam_index=3))
print ('Beam Search, k=5:', beam_search_predictions(im, beam_index=5))
print ('Beam Search, k=7:', beam_search_predictions(im, beam_index=7))
Image.open(im)

Flicker8k_Dataset/2542662402_d781dd7f7c.jpg


KeyError: '<start>'

In [144]:
im = test_img[int(np.random.randint(0, 1000, size=1))]
print (im)
print ('Normal Max search:', predict_captions(im))
print ('Beam Search, k=3:', beam_search_predictions(im, beam_index=3))
print ('Beam Search, k=5:', beam_search_predictions(im, beam_index=5))
print ('Beam Search, k=7:', beam_search_predictions(im, beam_index=7))
Image.open(im)

Flicker8k_Dataset/1982852140_56425fa7a2.jpg
Flicker8k_Dataset/1982852140_56425fa7a2.jpg


KeyError: '<start>'